In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, RobertaForSequenceClassification
from transformers import BertModel, DistilBertModel, RobertaModel
from transformers import BertTokenizer, DistilBertTokenizer, RobertaTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import pandas as pd
import random
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
seed_val = 42
k_folds = 10
num_classes = 2
max_length = 512
batch_size = 8
num_epochs = 20

In [ ]:
data = pd.read_csv('''Dataset path''')

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = data['text'].tolist()
        self.labels = [1 if deceptive == "deceptive" else 0 for deceptive in data['deceptive'].tolist()]
        data[['joy']] = data[['joy']].astype(int)
        self.joy_sentences = data['joy'].tolist()
        data[['joy_words_count']] = data[['joy_words_count']].astype(int)
        self.joy_words = data['joy_words_count'].tolist()
        data[['joy_character_count']] = data[['joy_character_count']].astype(int)
        self.joy_character = data['joy_character_count'].tolist()
        data[['joy_uppercase_words']] = data[['joy_uppercase_words']].astype(int)
        self.joy_uppercase = data['joy_uppercase_words'].tolist()
        data[['joy_exclamation_mark']] = data[['joy_exclamation_mark']].astype(int)
        self.joy_exclamation = data['joy_exclamation_mark'].tolist()
        data[['anger']] = data[['anger']].astype(int)
        self.anger_sentences = data['anger'].tolist()
        data[['anger_words_count']] = data[['anger_words_count']].astype(int)
        self.anger_words = data['anger_words_count'].tolist()
        data[['anger_character_count']] = data[['anger_character_count']].astype(int)
        self.anger_character = data['anger_character_count'].tolist()
        data[['anger_uppercase_words']] = data[['anger_uppercase_words']].astype(int)
        self.anger_uppercase = data['anger_uppercase_words'].tolist()
        data[['anger_exclamation_mark']] = data[['anger_exclamation_mark']].astype(int)
        self.anger_exclamation = data['anger_exclamation_mark'].tolist()
        data[['sadness']] = data[['sadness']].astype(int)
        self.sadness_sentences = data['sadness'].tolist()
        data[['sadness_words_count']] = data[['sadness_words_count']].astype(int)
        self.sadness_words = data['sadness_words_count'].tolist()
        data[['sadness_character_count']] = data[['sadness_character_count']].astype(int)
        self.sadness_character = data['sadness_character_count'].tolist()
        data[['sadness_uppercase_words']] = data[['sadness_uppercase_words']].astype(int)
        self.sadness_uppercase = data['sadness_uppercase_words'].tolist()
        data[['sadness_exclamation_mark']] = data[['sadness_exclamation_mark']].astype(int)
        self.sadness_exclamation = data['sadness_exclamation_mark'].tolist()
        data[['surprise']] = data[['surprise']].astype(int)
        self.surprise_sentences = data['surprise'].tolist()
        data[['surprise_words_count']] = data[['surprise_words_count']].astype(int)
        self.surprise_words = data['surprise_words_count'].tolist()
        data[['surprise_character_count']] = data[['surprise_character_count']].astype(int)
        self.surprise_character = data['surprise_character_count'].tolist()
        data[['surprise_uppercase_words']] = data[['surprise_uppercase_words']].astype(int)
        self.surprise_uppercase = data['surprise_uppercase_words'].tolist()
        data[['surprise_exclamation_mark']] = data[['surprise_exclamation_mark']].astype(int)
        self.surprise_exclamation = data['surprise_exclamation_mark'].tolist()
        data[['anticipation']] = data[['anticipation']].astype(int)
        self.anticipation_sentences = data['anticipation'].tolist()
        data[['anticipation_words_count']] = data[['anticipation_words_count']].astype(int)
        self.anticipation_words = data['anticipation_words_count'].tolist()
        data[['anticipation_character_count']] = data[['anticipation_character_count']].astype(int)
        self.anticipation_character = data['anticipation_character_count'].tolist()
        data[['anticipation_uppercase_words']] = data[['anticipation_uppercase_words']].astype(int)
        self.anticipation_uppercase = data['anticipation_uppercase_words'].tolist()
        data[['anticipation_exclamation_mark']] = data[['anticipation_exclamation_mark']].astype(int)
        self.anticipation_exclamation = data['anticipation_exclamation_mark'].tolist()

    def __len__(self):
        return(len(self.texts))

    def __getitem__(self,idx):
        text = self.texts[idx]
        label = self.labels[idx]
        joy_sentences = self.joy_sentences[idx]
        joy_words = self.joy_words[idx]
        joy_character = self.joy_character[idx]
        joy_uppercase = self.joy_uppercase[idx]
        joy_exclamation = self.joy_exclamation[idx]
        anger_sentences = self.anger_sentences[idx]
        anger_words = self.anger_words[idx]
        anger_character = self.anger_character[idx]
        anger_uppercase = self.anger_uppercase[idx]
        anger_exclamation = self.anger_exclamation[idx]
        sadness_sentences = self.sadness_sentences[idx]
        sadness_words = self.sadness_words[idx]
        sadness_character = self.sadness_character[idx]
        sadness_uppercase = self.sadness_uppercase[idx]
        sadness_exclamation = self.sadness_exclamation[idx]
        surprise_sentences = self.surprise_sentences[idx]
        surprise_words = self.surprise_words[idx]
        surprise_character = self.surprise_character[idx]
        surprise_uppercase = self.surprise_uppercase[idx]
        surprise_exclamation = self.surprise_exclamation[idx]
        anticipation_sentences = self.anticipation_sentences[idx]
        anticipation_words = self.anticipation_words[idx]
        anticipation_character = self.anticipation_character[idx]
        anticipation_uppercase = self.anticipation_uppercase[idx]
        anticipation_exclamation = self.anticipation_exclamation[idx]

        encoding = self.tokenizer.encode_plus(text,
                                              max_length = self.max_length,
                                              add_special_tokens = True,
                                              padding = 'max_length',
                                              truncation = True,
                                              return_attention_mask = True,
                                              return_tensors = 'pt',
                                              )

        return {'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'label': torch.tensor(label),
                'joy_sentences': torch.tensor(joy_sentences),
                'joy_words': torch.tensor(joy_words),
                'joy_character': torch.tensor(joy_character),
                'joy_uppercase': torch.tensor(joy_uppercase),
                'joy_exclamation': torch.tensor(joy_exclamation),
                'anger_sentences': torch.tensor(anger_sentences),
                'anger_words': torch.tensor(anger_words),
                'anger_character': torch.tensor(anger_character),
                'anger_uppercase': torch.tensor(anger_uppercase),
                'anger_exclamation': torch.tensor(anger_exclamation),
                'sadness_sentences': torch.tensor(sadness_sentences),
                'sadness_words': torch.tensor(sadness_words),
                'sadness_character': torch.tensor(sadness_character),
                'sadness_uppercase': torch.tensor(sadness_uppercase),
                'sadness_exclamation': torch.tensor(sadness_exclamation),
                'surprise_sentences': torch.tensor(surprise_sentences),
                'surprise_words': torch.tensor(surprise_words),
                'surprise_character': torch.tensor(surprise_character),
                'surprise_uppercase': torch.tensor(surprise_uppercase),
                'surprise_exclamation': torch.tensor(surprise_exclamation),
                'anticipation_sentences': torch.tensor(anticipation_sentences),
                'anticipation_words': torch.tensor(anticipation_words),
                'anticipation_character': torch.tensor(anticipation_character),
                'anticipation_uppercase': torch.tensor(anticipation_uppercase),
                'anticipation_exclamation': torch.tensor(anticipation_exclamation),
                }

In [ ]:
class classifier(nn.Module):
    def __init__(self, bert_model_name, num_classes, dropout_val):
        super(classifier, self).__init__()
        self.fModel = sModel.from_pretrained(model_name)
        self.joy_sentences_feature = nn.Embedding(96,2)
        self.joy_words_feature = nn.Embedding(1536,2)
        self.joy_character_feature = nn.Embedding(6144,2)
        self.joy_uppercase_feature = nn.Embedding(96,2)
        self.joy_exclamation_feature = nn.Embedding(24,2)
        self.anger_sentences_feature = nn.Embedding(96,2)
        self.anger_words_feature = nn.Embedding(1536,2)
        self.anger_character_feature = nn.Embedding(6144,2)
        self.anger_uppercase_feature = nn.Embedding(96,2)
        self.anger_exclamation_feature = nn.Embedding(24,2)
        self.sadness_sentences_feature = nn.Embedding(96,2)
        self.sadness_words_feature = nn.Embedding(1536,2)
        self.sadness_character_feature = nn.Embedding(6144,2)
        self.sadness_uppercase_feature = nn.Embedding(96,2)
        self.sadness_exclamation_feature = nn.Embedding(24,2)
        self.surprise_sentences_feature = nn.Embedding(96,2)
        self.surprise_words_feature = nn.Embedding(1536,2)
        self.surprise_character_feature = nn.Embedding(6144,2)
        self.surprise_uppercase_feature = nn.Embedding(96,2)
        self.surprise_exclamation_feature = nn.Embedding(24,2)
        self.anticipation_sentences_feature = nn.Embedding(96,2)
        self.anticipation_words_feature = nn.Embedding(1536,2)
        self.anticipation_character_feature = nn.Embedding(6144,2)
        self.anticipation_uppercase_feature = nn.Embedding(96,2)
        self.anticipation_exclamation_feature = nn.Embedding(24,2)
        self.dropout = nn.Dropout(dropout_val)
        self.classifier = nn.Linear(768 + 50 , num_classes)
    
    def forward(self, input_ids, attention_mask, joy_sentences, joy_words, joy_character, joy_uppercase, joy_exclamation,
                anger_sentences, anger_words, anger_character, anger_uppercase, anger_exclamation, sadness_sentences,
                sadness_words, sadness_character, sadness_uppercase, sadness_exclamation, surprise_sentences,
                surprise_words, surprise_character, surprise_uppercase, surprise_exclamation, anticipation_sentences,
                anticipation_words, anticipation_character, anticipation_uppercase, anticipation_exclamation):
        outputs = self.fModel(input_ids = input_ids, attention_mask = attention_mask)
        pooler = outputs[0][:, 0]
        joy_sentences_embedding = self.joy_sentences_feature(joy_sentences)
        joy_words_embedding = self.joy_words_feature(joy_words)
        joy_character_embedding = self.joy_character_feature(joy_character)
        joy_uppercase_embedding = self.joy_uppercase_feature(joy_uppercase)
        joy_exclamation_embedding = self.joy_exclamation_feature(joy_exclamation)
        anger_sentences_embedding = self.anger_sentences_feature(anger_sentences)
        anger_words_embedding = self.anger_words_feature(anger_words)
        anger_character_embedding = self.anger_character_feature(anger_character)
        anger_uppercase_embedding = self.anger_uppercase_feature(anger_uppercase)
        anger_exclamation_embedding = self.anger_exclamation_feature(anger_exclamation)
        sadness_sentences_embedding = self.sadness_sentences_feature(sadness_sentences)
        sadness_words_embedding = self.sadness_words_feature(sadness_words)
        sadness_character_embedding = self.sadness_character_feature(sadness_character)
        sadness_uppercase_embedding = self.sadness_uppercase_feature(sadness_uppercase)
        sadness_exclamation_embedding = self.sadness_exclamation_feature(sadness_exclamation)
        surprise_sentences_embedding = self.surprise_sentences_feature(surprise_sentences)
        surprise_words_embedding = self.surprise_words_feature(surprise_words)
        surprise_character_embedding = self.surprise_character_feature(surprise_character)
        surprise_uppercase_embedding = self.surprise_uppercase_feature(surprise_uppercase)
        surprise_exclamation_embedding = self.surprise_exclamation_feature(surprise_exclamation)
        anticipation_sentences_embedding = self.anticipation_sentences_feature(anticipation_sentences)
        anticipation_words_embedding = self.anticipation_words_feature(anticipation_words)
        anticipation_character_embedding = self.anticipation_character_feature(anticipation_character)
        anticipation_uppercase_embedding = self.anticipation_uppercase_feature(anticipation_uppercase)
        anticipation_exclamation_embedding = self.anticipation_exclamation_feature(anticipation_exclamation)
        pooler = torch.cat((pooler, joy_sentences_embedding, joy_words_embedding, joy_character_embedding, joy_uppercase_embedding,
                            joy_exclamation_embedding, anger_sentences_embedding, anger_words_embedding, anger_character_embedding,
                            anger_uppercase_embedding, anger_exclamation_embedding, sadness_sentences_embedding, sadness_words_embedding,
                            sadness_character_embedding, sadness_uppercase_embedding, sadness_exclamation_embedding, 
                            surprise_sentences_embedding, surprise_words_embedding, surprise_character_embedding,
                            surprise_uppercase_embedding, surprise_exclamation_embedding, anticipation_sentences_embedding,
                            anticipation_words_embedding, anticipation_character_embedding, anticipation_uppercase_embedding,
                            anticipation_exclamation_embedding,), dim = -1)
        pooler = nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        logits = self.classifier(pooler)
        return logits

In [ ]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    losses = []
    predictions = []
    actual_labels = []

    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        joy_sentences = batch['joy_sentences'].to(device)
        joy_words = batch['joy_words'].to(device)
        joy_character = batch['joy_character'].to(device)
        joy_uppercase = batch['joy_uppercase'].to(device)
        joy_exclamation = batch['joy_exclamation'].to(device)
        anger_sentences = batch['anger_sentences'].to(device)
        anger_words = batch['anger_words'].to(device)
        anger_character = batch['anger_character'].to(device)
        anger_uppercase = batch['anger_uppercase'].to(device)
        anger_exclamation = batch['anger_exclamation'].to(device)
        sadness_sentences = batch['sadness_sentences'].to(device)
        sadness_words = batch['sadness_words'].to(device)
        sadness_character = batch['sadness_character'].to(device)
        sadness_uppercase = batch['sadness_uppercase'].to(device)
        sadness_exclamation = batch['sadness_exclamation'].to(device)
        surprise_sentences = batch['surprise_sentences'].to(device)
        surprise_words = batch['surprise_words'].to(device)
        surprise_character = batch['surprise_character'].to(device)
        surprise_uppercase = batch['surprise_uppercase'].to(device)
        surprise_exclamation = batch['surprise_exclamation'].to(device)
        anticipation_sentences = batch['anticipation_sentences'].to(device)
        anticipation_words = batch['anticipation_words'].to(device)
        anticipation_character = batch['anticipation_character'].to(device)
        anticipation_uppercase = batch['anticipation_uppercase'].to(device)
        anticipation_exclamation = batch['anticipation_exclamation'].to(device)
        outputs = model(input_ids = input_ids, attention_mask = attention_mask, joy_sentences = joy_sentences, joy_words = joy_words,
                        joy_character = joy_character, joy_uppercase = joy_uppercase, joy_exclamation = joy_exclamation,
                        anger_sentences = anger_sentences, anger_words = anger_words, anger_character = anger_character,
                        anger_uppercase = anger_uppercase, anger_exclamation = anger_exclamation, sadness_sentences = sadness_sentences,
                        sadness_words = sadness_words, sadness_character = sadness_character, sadness_uppercase = sadness_uppercase,
                        sadness_exclamation = sadness_exclamation, surprise_sentences = surprise_sentences, surprise_words = surprise_words,
                        surprise_character = surprise_character, surprise_uppercase = surprise_uppercase,
                        surprise_exclamation = surprise_exclamation, anticipation_sentences = anticipation_sentences,
                        anticipation_words = anticipation_words, anticipation_character = anticipation_character,
                        anticipation_uppercase = anticipation_uppercase, anticipation_exclamation = anticipation_exclamation)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        losses.append(loss.item())
        _, preds = torch.max(outputs, dim=1)
        predictions.extend(preds.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = np.mean(losses)
    return avg_loss, predictions, actual_labels

In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            joy_sentences = batch['joy_sentences'].to(device)
            joy_words = batch['joy_words'].to(device)
            joy_character = batch['joy_character'].to(device)
            joy_uppercase = batch['joy_uppercase'].to(device)
            joy_exclamation = batch['joy_exclamation'].to(device)
            anger_sentences = batch['anger_sentences'].to(device)
            anger_words = batch['anger_words'].to(device)
            anger_character = batch['anger_character'].to(device)
            anger_uppercase = batch['anger_uppercase'].to(device)
            anger_exclamation = batch['anger_exclamation'].to(device)
            sadness_sentences = batch['sadness_sentences'].to(device)
            sadness_words = batch['sadness_words'].to(device)
            sadness_character = batch['sadness_character'].to(device)
            sadness_uppercase = batch['sadness_uppercase'].to(device)
            sadness_exclamation = batch['sadness_exclamation'].to(device)
            surprise_sentences = batch['surprise_sentences'].to(device)
            surprise_words = batch['surprise_words'].to(device)
            surprise_character = batch['surprise_character'].to(device)
            surprise_uppercase = batch['surprise_uppercase'].to(device)
            surprise_exclamation = batch['surprise_exclamation'].to(device)
            anticipation_sentences = batch['anticipation_sentences'].to(device)
            anticipation_words = batch['anticipation_words'].to(device)
            anticipation_character = batch['anticipation_character'].to(device)
            anticipation_uppercase = batch['anticipation_uppercase'].to(device)
            anticipation_exclamation = batch['anticipation_exclamation'].to(device)
            outputs = model(input_ids = input_ids, attention_mask = attention_mask, joy_sentences = joy_sentences, joy_words = joy_words,
                            joy_character = joy_character, joy_uppercase = joy_uppercase, joy_exclamation = joy_exclamation,
                            anger_sentences = anger_sentences, anger_words = anger_words, anger_character = anger_character,
                            anger_uppercase = anger_uppercase, anger_exclamation = anger_exclamation, sadness_sentences = sadness_sentences,
                            sadness_words = sadness_words, sadness_character = sadness_character, sadness_uppercase = sadness_uppercase,
                            sadness_exclamation = sadness_exclamation, surprise_sentences = surprise_sentences,
                            surprise_words = surprise_words, surprise_character = surprise_character, surprise_uppercase = surprise_uppercase,
                            surprise_exclamation = surprise_exclamation, anticipation_sentences = anticipation_sentences,
                            anticipation_words = anticipation_words, anticipation_character = anticipation_character,
                            anticipation_uppercase = anticipation_uppercase, anticipation_exclamation = anticipation_exclamation)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            losses.append(loss.item())
            _, preds = torch.max(outputs, dim = 1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    avg_loss = np.mean(losses)
    return avg_loss, predictions, actual_labels

In [ ]:
# sModels = [BertModel, RobertaModel, DistilBertModel]
sModels = [DistilBertModel]
# model_list = ['bert-base-uncased', 'roberta-base', 'distilbert-base-uncased']
model_list = ['distilbert-base-uncased']
# tokenizer_list = [BertTokenizer, RobertaTokenizer, DistilBertTokenizer]
tokenizer_list = [DistilBertTokenizer]
# dropout_list = [0.2, 0.3, 0.4]
dropout_list = [0.3]
# learning_rate_list = [1e-5, 3e-5, 5e-5, 7e-5, 9e-5]
learning_rate_list = [1e-5, 3e-5, 5e-5]

training_stats = []
best_testing_stats = []
last_testing_stats = []

kf = StratifiedKFold(n_splits = k_folds, shuffle = True, random_state = seed_val)

for i in range(len(model_list)):
    for dropout in dropout_list:
        for learning_rate in learning_rate_list:
            sModel = sModels[i]
            model_name = model_list[i]
            tokenizer_name = tokenizer_list[i]

            tokenizer = tokenizer_name.from_pretrained(model_name, do_lower_case = True)
            dataset = TextClassificationDataset(data, tokenizer, max_length)

            average_accuracy = 0
            average_precision = 0
            average_recall = 0
            average_f1_score = 0
            average_roc_auc = 0

            average_best_accuracy = 0
            average_best_precision = 0
            average_best_recall = 0
            average_best_f1_score = 0
            average_best_roc_auc = 0

            for fold, (train_idx, test_idx) in enumerate(kf.split(data['text'], data['deceptive'])):
                random.seed(seed_val)
                np.random.seed(seed_val)
                torch.manual_seed(seed_val)
                torch.cuda.manual_seed_all(seed_val)
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False
                os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
                
                train_dataset = Subset(dataset, train_idx)
                val_dataset = Subset(dataset, test_idx)
                train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
                test_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                model = classifier(model_name, num_classes, dropout).to(device)
                optimizer = AdamW(model.parameters(), lr = learning_rate)
                total_steps = len(train_dataloader) * num_epochs
                scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

                temp_accuracy_score = 0
                temp_precision_score = 0
                temp_recall_score = 0
                temp_f1_score = 0
                temp_roc_auc = 0

                for epoch in range(num_epochs):
                    training_loss, training_predictions, training_actual_labels = train(model, train_dataloader, optimizer, scheduler, device)
                    training_accuracy_score = accuracy_score(training_actual_labels, training_predictions)

                    test_loss, test_predictions, test_actual_labels = evaluate(model, test_dataloader, device)
                    test_accuracy_score = accuracy_score(test_actual_labels, test_predictions)
                    test_precision_score = precision_score(test_actual_labels, test_predictions)
                    test_recall_score = recall_score(test_actual_labels, test_predictions)
                    test_f1_score = f1_score(test_actual_labels, test_predictions, average = 'macro')
                    test_roc_auc = roc_auc_score(test_actual_labels, test_predictions)

                    print(f"Model: {model_name}, Dropout: {dropout}, Learning rate: {learning_rate}, Fold: {fold + 1}, Epoch: {epoch + 1}, Training Accuracy: {training_accuracy_score:.4f}, Test Accuracy: {test_accuracy_score:.4f}")

                    training_stats.append({'Model': model_name,
                                           'Dropout': dropout,
                                           'Learning Rate': learning_rate,
                                           'Fold': fold + 1,
                                           'Epoch': epoch + 1,
                                           'Training Loss': training_loss,
                                           'Test Loss': test_loss,
                                           'Training Accuracy': training_accuracy_score,
                                           'Test Accuracy': test_accuracy_score,
                                           'Test Precision': test_precision_score,
                                           'Test Recall': test_recall_score,
                                           'Test F-Score': test_f1_score,
                                           'Test Roc Auc': test_roc_auc,
                                           })
                    if test_accuracy_score > temp_accuracy_score:
                        temp_accuracy_score = test_accuracy_score
                    if test_precision_score > temp_precision_score:
                        temp_precision_score = test_precision_score
                    if test_recall_score > temp_recall_score:
                        temp_recall_score = test_recall_score
                    if test_f1_score > temp_f1_score:
                        temp_f1_score = test_f1_score
                    if test_roc_auc > temp_roc_auc:
                        temp_roc_auc = test_roc_auc
                    
                    if epoch + 1 == num_epochs:
                        average_accuracy += test_accuracy_score
                        average_precision += test_precision_score
                        average_recall += test_recall_score
                        average_f1_score += test_f1_score
                        average_roc_auc += test_roc_auc

                        average_best_accuracy += temp_accuracy_score
                        average_best_precision += temp_precision_score
                        average_best_recall += temp_recall_score
                        average_best_f1_score += temp_f1_score
                        average_best_roc_auc += temp_roc_auc

            print(f"Best Test Accuracy: {average_best_accuracy:.4f}, Last Test Accuracy: {average_accuracy:.4f}")

            last_testing_stats.append({'Model': model_name,
                                       'Dropout': dropout,
                                       'Learning Rate': learning_rate,
                                       'Test Accuracy': average_accuracy / k_folds,
                                       'Test Precision': average_precision / k_folds,
                                       'Test Recall': average_recall / k_folds,
                                       'Test F-Score': average_f1_score / k_folds,
                                       'Test Roc Auc': average_roc_auc / k_folds,
                                       })
            best_testing_stats.append({'Model': model_name,
                                       'Dropout': dropout,
                                       'Learning Rate': learning_rate,
                                       'Test Accuracy': average_best_accuracy / k_folds,
                                       'Test Precision': average_best_precision / k_folds,
                                       'Test Recall': average_best_recall / k_folds,
                                       'Test F-Score': average_best_f1_score / k_folds,
                                       'Test Roc Auc': average_best_roc_auc / k_folds,
                                       })

In [ ]:
train_stats = pd.DataFrame(data = training_stats)
best_test_stats = pd.DataFrame(data = best_testing_stats)
last_test_stats = pd.DataFrame(data = last_testing_stats)

In [ ]:
train_stats.to_csv('exp4_train_6.csv', index = False, float_format='%.5f')
best_test_stats.to_csv('exp4_test_best_6.csv', index = False, float_format='%.5f')
last_test_stats.to_csv('exp4_test_last_6.csv', index = False, float_format='%.5f')